In [ ]:
import os
import pandas as pd
import json
import glob
from PIL import Image
import numpy as np
import pickle
import cv2
import mmcv
import random
import matplotlib.pyplot as plt

In [ ]:
random.seed(42)

pics = os.listdir('data/train_annotations')
random.shuffle(pics)
train = pics[:170]
val = pics[170:]
len(train), len(val), 

In [ ]:
labels = os.listdir('data/master_images/')
labels = sorted([x[:-4] for x in labels])
labels = dict(zip(labels, range(len(labels))))
with open('data/remap_dict.json', 'w') as handle:
        json.dump(labels, handle)
        print('remap_dict ready')

In [ ]:
def annot(folder=train):
    ll = []
    annot_path = './data/train_annotations'
    images_path = './data/train_images'
    save_path = './data/resized_images'
    with open('./data/remap_dict.json', 'r') as f:
        remap_dict = json.load(f)
    annotations_result = []


    all_ids = os.listdir(annot_path)
    for _file in folder:
        _fid = _file.split('.')[0]
        image_path = os.path.join(images_path, '{}.jpg'.format(_fid))
        image = cv2.imread(image_path)

        h, w, c = image.shape
        i = 0
        #нарезаю рисунки на квадраты 816px с отступом друг от друга на 408px
        for ww in range(w//408):
            for hh in range(h//408):
                w0 = ww*408
                h0 = hh*408
                h1 = h0+816
                w1 = w0+816
                if w1>w or h1>h:
                    break
                image1 = image[h0:h1, w0:w1]
                file_path = os.path.join(annot_path, _file)

                annot_instance = {}
                annot_instance['filename'] = 'data/resized_images/{}_{:02d}.jpg'.format(_fid, i)
                annot_instance['height'] = image1.shape[0]
                annot_instance['width'] = image1.shape[1]
                annot_instance['ann'] = {}

                with open(file_path, 'r') as f:
                    annot_data = json.load(f)
                boxes = []
                labels = []

                for label in annot_data['labels']:
                    class_label = label['category']
                    x1 = int(label['box2d']['x1'])
                    x2 = int(label['box2d']['x2'])
                    y1 = int(label['box2d']['y1'])
                    y2 = int(label['box2d']['y2'])
                    if w0<=x1<=w1 and w0<=x2<=w1 and h0<=y1<=h1 and h0<=y2<=h1:
                        boxes.append([x1-w0, y1-h0, x2-w0, y2-h0])
                        class_label = remap_dict[str(class_label)]
                        labels.append(class_label)
                # записываю аннотации и картинки только с коробками, пустые пропускаю
                if len(labels)>0:
                    cv2.imwrite(os.path.join(save_path, '{}_{:02d}.jpg'.format(_fid, i)), image1)
                    annot_instance['ann']['bboxes'] = (np.array(boxes)).astype(np.float32)
                    annot_instance['ann']['labels'] = (np.array(labels)).astype(np.int64)
                    annot_instance['ann']['bboxes_ignore'] = np.zeros((0, 4), dtype=np.float32)
                    annot_instance['ann']['labels_ignore'] = (np.array([])).astype(np.int64)
                    annotations_result.append(annot_instance)
                i+=1
    if folder == train:
        fol = 'train'
    else: 
        fol = 'val'
    mmcv.dump(annotations_result, './data/'+fol+'_mmannotations.pkl')
    print('annot ready')

In [ ]:
annot(val)
annot(train)

In [ ]:
with open('data/val_mmannotations.pkl', 'rb') as handle:
        unpickler = pickle.Unpickler(handle)
        data = unpickler.load()

In [ ]:
data[55]['filename'], data[55]['ann']['bboxes']

In [ ]:
for i in range(10):
    img = cv2.imread(data[i]['filename'])
    for box in data[i]['ann']['bboxes']:
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (x1, y1, x2-x1, y2-y1), 3, 3)
    plt.imshow(img)
    plt.axis('off')
    plt.show()